In [1]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 m2m모델로 만든 다국어 번역기 입니다."
chinese_text = "生活就像一盒巧克力。"

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 한국어를 영어로
tokenizer.src_lang = "ko"
encoded_hi = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
result1 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result1)

# translate Chinese to English
tokenizer.src_lang = "zh"
encoded_zh = tokenizer(chinese_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("ko"))
result2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result2)

['This is a multi-language translator made with the m2m model.']
['인생은 초콜릿 상자와 같다.']


In [3]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load model & tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# Translation function
def translate(text, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang)
    )
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return result[0]


# Gradio interface
def translate_ui(text, lang_option):
    if lang_option == "한국어 → 영어":
        return translate(text, "ko", "en")
    elif lang_option == "중국어 → 한국어":
        return translate(text, "zh", "ko")
    elif lang_option == "한국어 → 일본어":
        return translate(text, "ko", "ja")

title = "🌏 M2M100 다국어 번역기"
description = "한국어→영어 / 중국어→한국어/ 한국어 → 일본어 번역을 지원합니다."

demo = gr.Interface(
    fn=translate_ui,
    inputs=[
        gr.Textbox(label="입력 텍스트를 넣으세요"),
        gr.Radio(
            ["한국어 → 영어", "중국어 → 한국어","한국어 → 일본어"],
            label="번역 방향 선택"
        )
    ],
    outputs=gr.Textbox(label="번역 결과"),
    title=title,
    description=description
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [6]:
demo.close()

Closing server running on port: 7861


# 한영-영한 번역후 음성으로 읽어주는 서비스

영어를 읽어주는 모델

In [8]:
!pip install -q kokoro>=0.9.2 soundfile
#!apt-get -qq -y install espeak-ng > /dev/null 2>&1
from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
import torch
pipeline = KPipeline(lang_code='a')
text = "Politicians are joining in with the criticisms made by some Korean football fans against Jurgen Klinsmann, the national team's head coach who 1) has taken flak for his 2) tactics and reactions after Korea's 3) semifinal loss in the 2023 AFC Asian Cup hosted by Qatar."
generator = pipeline(text, voice='af_heart')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)


config.json: 0.00B [00:00, ?B/s]

/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


kokoro-v1_0.pth:   0%|          | 0.00/327M [00:00<?, ?B/s]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.3 MB/s  0:00:00m0:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


voices/af_heart.pt:   0%|          | 0.00/523k [00:00<?, ?B/s]

0 Politicians are joining in with the criticisms made by some Korean football fans against Jurgen Klinsmann, the national team's head coach who 1) has taken flak for his 2) tactics and reactions after Korea's 3) semifinal loss in the 2023 AFC Asian Cup hosted by Qatar. pˌɑlətˈɪʃənz ɑɹ ʤˈYnɪŋ ˈɪn wɪð ðə kɹˈɪTəsˌɪzəmz mˌAd bI sˌʌm kəɹˈiən fˈʊtbˌɔl fˈænz əɡˈɛnst ʤˈɜɹʤən klˈɪnsmən, ðə nˈæʃənᵊl tˈimz hˈɛd kˈOʧ hˌu wˈʌn) hæz tˈAkən flˈæk fɔɹ hɪz tˈu) tˈæktɪks ænd ɹiˈækʃənz ˈæftəɹ kəɹˈiəz θɹˈi) sˌɛmˌIfˈInᵊl lˈɔs ɪn ðə twˈɛnti twˈɛnti θɹˈi ˌAˌɛfsˈi ˈAʒən kˈʌp hˈOstᵻd bI kˈɑtˌɑɹ.


위에 코드를 실행하면 음성파일도 같이 생성된다

한국어를 읽어주는 모델

In [12]:
#!pip install uroman

In [21]:
from transformers import VitsModel, AutoTokenizer
import torch
from uroman import Uroman
import soundfile as sf

model = VitsModel.from_pretrained("facebook/mms-tts-kor")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-kor")

text = "이것은 허깅페이스에서 받는 모델로 만든 한국어 읽어주는 프로그램 입니다."
uroman = Uroman()
roman_text = uroman.romanize_string(text)

inputs = tokenizer(roman_text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform
waveform = output.squeeze(0).cpu().numpy()   # shape (N,)
    
display(Audio(waveform, rate=model.config.sampling_rate))
sf.write("krorean_voice.wav", waveform,24000)

통합

In [1]:

import gradio as gr
import torch
import soundfile as sf
import numpy as np

from transformers import (
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    VitsModel,
    AutoTokenizer,
)
from kokoro import KPipeline   # Kokoro-82M 파이썬 래퍼
from uroman import Uroman      # pip install uroman

# -----------------------------
# 0. 디바이스 (GPU 우선)
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------------
# 1. 번역 모델 (M2M100)
# -----------------------------
translation_model_name = "facebook/m2m100_418M"
translation_model = M2M100ForConditionalGeneration.from_pretrained(
    translation_model_name
).to(device)
translation_tokenizer = M2M100Tokenizer.from_pretrained(translation_model_name)

# -----------------------------
# 2. 영어 TTS: Kokoro-82M
# -----------------------------
kokoro_pipeline = KPipeline(lang_code="a")

def tts_english_kokoro(text: str, path: str = "tts_en.wav") -> str | None:
    text = (text or "").strip()
    if not text:
        return None

    generator = kokoro_pipeline(text, voice="af_heart")

    chunks = []
    for _, _, audio in generator:
        chunks.append(audio)

    if not chunks:
        return None

    audio_cat = np.concatenate(chunks)
    sf.write(path, audio_cat, 24000)
    return path

# -----------------------------
# 3. 한국어 TTS: MMS-TTS-KOR (VITS) + Uroman
# -----------------------------
kor_tts_name = "facebook/mms-tts-kor"
kor_tts_model = VitsModel.from_pretrained(kor_tts_name).to(device)
kor_tts_tokenizer = AutoTokenizer.from_pretrained(kor_tts_name)

uroman = Uroman()   # 한 번만 초기화

def tts_korean_mms(text: str, path: str = "tts_ko.wav") -> str | None:
    text = (text or "").strip()
    if not text:
        return None

    # 1) Uroman으로 로마자 변환
    roman_text = uroman.romanize_string(text)

    # 2) 토크나이저 입력 생성
    inputs = kor_tts_tokenizer(roman_text, return_tensors="pt").to(device)

    # 3) 모델 인퍼런스
    with torch.no_grad():
        outputs = kor_tts_model(**inputs).waveform

    wav = outputs.squeeze().cpu().numpy()
    if wav.size == 0:
        return None

    # 4) WAV 파일로 저장
    sf.write(path, wav, samplerate=kor_tts_model.config.sampling_rate)
    return path

# -----------------------------
# 4. 언어 매핑
# -----------------------------
LANGUAGES = {
    "Korean": "ko",
    "English": "en",
    "Chinese": "zh",
    "Japanese": "ja",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Hindi": "hi",
}

# -----------------------------
# 5. 번역 + TTS 함수
# -----------------------------
def translate_and_speak(text, src_lang_name, tgt_lang_name):
    text = text or ""
    if not text.strip():
        return "", None

    src_lang = LANGUAGES[src_lang_name]
    tgt_lang = LANGUAGES[tgt_lang_name]

    # ----- 번역 -----
    if src_lang == tgt_lang:
        translated_text = text.strip()
    else:
        translation_tokenizer.src_lang = src_lang
        encoded = translation_tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
        ).to(device)

        with torch.no_grad():
            generated_tokens = translation_model.generate(
                **encoded,
                forced_bos_token_id=translation_tokenizer.get_lang_id(tgt_lang),
                max_length=512,
            )

        translated_text = translation_tokenizer.batch_decode(
            generated_tokens, skip_special_tokens=True
        )[0]

    # ----- TTS -----
    audio_path = None
    if tgt_lang == "en":
        audio_path = tts_english_kokoro(translated_text)
    elif tgt_lang == "ko":
        audio_path = tts_korean_mms(translated_text)

    return translated_text, audio_path

# -----------------------------
# 6. Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🌐 다국어 번역 + TTS (영어 Kokoro + 한국어 MMS)")

    with gr.Row():
        src_lang = gr.Dropdown(
            list(LANGUAGES.keys()), label="원본 언어", value="Korean"
        )
        tgt_lang = gr.Dropdown(
            list(LANGUAGES.keys()), label="번역 언어", value="English"
        )

    with gr.Row():
        input_text = gr.Textbox(lines=8, label="입력 텍스트")
        output_text = gr.Textbox(lines=8, label="번역 결과", interactive=False)

    audio_output = gr.Audio(label="음성 출력", type="filepath")

    translate_button = gr.Button("번역 + 음성 생성")
    translate_button.click(
        fn=translate_and_speak,
        inputs=[input_text, src_lang, tgt_lang],
        outputs=[output_text, audio_output],
    )

    demo.queue()

# 주피터 노트북에서는 demo.launch()를 마지막 셀에서 직접 호출하는 것을 권장합니다.
print("모델 로딩 및 Gradio 앱 정의 완료. 다음 셀에서 demo.launch()를 실행하세요.")


Using device: cuda


/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


모델 로딩 및 Gradio 앱 정의 완료. 다음 셀에서 demo.launch()를 실행하세요.


In [2]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [3]:
demo.close()

Closing server running on port: 7860
